In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import os
import pandas as pd
import time
import cv2
import pytesseract
from os import path
from IPython.display import clear_output
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

In [2]:
# This process takes a long time to run
words = []
label = []
filenames = []
# Pro pool
for file in os.listdir("images/nonanti_pool_63432"):
    start, end = os.path.splitext(file)
    if end == ".txt":
        os.remove("images/nonanti_pool_63432/"+file) # Remove text files from data folder
    else:
        imagepath = "images/nonanti_pool_63432/"+file
        image = cv2.imread(imagepath)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray = cv2.medianBlur(gray, 3)
        newimage = "greyimage.png"
        cv2.imwrite(newimage, gray)
        text = pytesseract.image_to_string(Image.open(newimage))
        # Uncomment these lines to show each image and its respective text
        #plt.figure()
        #plt.imshow(gray, cmap='gray')
        #plt.show()
        #print(text)
        if text != "":
            words.append(text)
            label.append("Pro")
            filenames.append(file)

# Anti pool
for file in os.listdir("images/anti_pool_61696"):
    start, end = os.path.splitext(file)
    if end == ".txt":
        os.remove("images/anti_pool_61696/"+file) # Remove text files from data folder
    else:
        imagepath = "images/anti_pool_61696/"+file
        image = cv2.imread(imagepath)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray = cv2.medianBlur(gray, 3)
        newimage = "greyimage.png"
        cv2.imwrite(newimage, gray)
        text = pytesseract.image_to_string(Image.open(newimage))
        # Uncomment these lines to show each image and its respective text
        #plt.figure()
        #plt.imshow(gray, cmap='gray')
        #plt.show()
        #print(text)
        if text != "":
            words.append(text)
            label.append("Anti")
            filenames.append(file)
            
realData = pd.DataFrame({"File":filenames, "Text":words, "Class":label})
realData.head()

,File,Text,Class
0,cdcglobal-20170120-514+BPfN66MlEgc+-1.jpg,The world is more\n\nconnected than in\n\nany ...,Pro
1,cdcglobal-20170203-503+BQD_bDngXHk+-1.jpg,FEBRUARY 4\n\nTHE GLOBAL CANCER EPIDEMIC\nCONT...,Pro
2,cdcglobal-20170204-502+BQGBCsaA9We+-1.jpg,As a global public health community we can wor...,Pro
3,cdcglobal-20170322-472+BR8UovohX6R+-1.jpg,OVER 80% OF OUR WASTEWATER\nFLOWS BACK TO NATU...,Pro
4,cdcglobal-20170420-448+BTHGEW6B2hj+-1.jpg,( More than\n\n(m1) 88 0e)\\n\npeople are affe...,Pro


In [3]:
#realData.to_csv("RealData.csv")

In [2]:
realData = pd.read_csv("RealData.csv")
realData.head()
realData.shape

(4136, 4)

In [3]:
# Code to perform manual pytesseract accuracy test of 100 random images, 50 pro, 50 anti
ProPool = realData[realData["Class"] == "Pro"].sample(50, random_state = 3)
AntiPool = realData[realData["Class"] ==  "Anti"].sample(50, random_state=3)
pool = ProPool.append(AntiPool)
pytext = []

for file in pool["File"]:
    imagepath = "images/nonanti_pool_63432/"+file
    if not path.exists(imagepath):
        imagepath = "images/anti_pool_61696/"+file
    image = cv2.imread(imagepath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray, 3)
    newimage = "greyimage.png"
    cv2.imwrite(newimage, gray)
    text = pytesseract.image_to_string(Image.open(newimage))
    #plt.figure(figsize=(10,10))
    #plt.clf()
    #plt.imshow(gray, cmap='gray')
    #plt.show()
    print("PyTesseract Output: \n" + text)
    pytext.append(text)
    #input("Press Enter When Ready")
    clear_output(wait=True)

PyTesseract Output: 
WHEN SOMEONE SAYS THE GOVERNMENT

LIES ABOUT EVERYTHING BUT WOULDN'T
LIE ABOUT THE SHAPE OF THE EARTH


In [4]:
paths = []

for file in pool["File"]:
    imagepath = "images/nonanti_pool_63432/"+file
    if not path.exists(imagepath):
        imagepath = "images/anti_pool_61696/"+file
    paths.append(imagepath)

In [5]:
pystrings = pd.read_excel("PyTesseractStringRating.xlsx")
pystrings["TesseractResult"] = pd.Series(pytext)
pystrings["ImagePath"] = pd.Series(paths)
pystrings.head()
#pystrings.to_excel("PytesseractStringRatingComplete.xlsx")

,Actual String,TesseractResult,ImagePath
0,Are you a pleural effusion? Because I can't br...,OAs SAV OOo Sa AMAA,images/nonanti_pool_63432/dr.majestic_md-20180...
1,"My gynecologist follows me on instagram, I rea...",CoE Ear |]\ny) @chashmaswag\n\nMy gynecologist...,images/nonanti_pool_63432/daily_medicine-20190...
2,You were born for greatness.,@schoolacceptance eon\n\nYou were born\nfor gr...,images/nonanti_pool_63432/surviving.medicine-2...
3,Comment Below Your thought of the day,Oma,images/nonanti_pool_63432/daily_medicine-20180...
4,Diabetes except it doesn’t when will the sugar...,DIABETES:\n\nen\n\noe)\n\n \n\n“piictgdagn | t...,images/nonanti_pool_63432/drjoshuawolrich-2018...


In [ ]:
data = pystrings
ratings = []
for i in range(data["Actual String"].size):
    print("Actual String: ", data["Actual String"][i])
    print("\n")
    print("Pytesseract String: ", data["TesseractResult"][i])
    ratings.append(input("Rating: "))
    clear_output(wait=True)
    
data["Rating"] = pd.Series(ratings)
#data.to_excel("PytesseractStringRatingComplete.xlsx")

Actual String:  Are you a pleural effusion? Because I can't breathe when you're around


Pytesseract String:  OAs SAV OOo Sa AMAA
